## Import Libraries

In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.vit import ViT

In [2]:
print(f"Torch: {torch.__version__}")

Torch: 1.11.0


In [3]:
# Training settings
batch_size = 16
epochs = 30
lr = 3e-5
gamma = 0.7
seed = 42

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [5]:
device = 'cuda'

## Load Data

In [6]:
PATH_none_crash_prep = '../../dataset/data_preprocessed/None-crash'
PATH_vulner_prep = '../../dataset/data_preprocessed/Vulner'

none_crash_drive_list = glob.glob(PATH_none_crash_prep + "/*")
vulner_drive_list = glob.glob(PATH_vulner_prep + "/*")

In [7]:
none_crash_data_list = list()
vulner_data_list = list()

for PATH_drive in none_crash_drive_list:
    none_crash_data_list += glob.glob(PATH_drive + "/*")

for PATH_drive in vulner_drive_list:
    vulner_data_list += glob.glob(PATH_drive + "/*")

In [8]:
print(f"None-crash: {len(none_crash_data_list)}")
print(f"Vulner: {len(vulner_data_list)}")

None-crash: 30498
Vulner: 3788


In [9]:
none_crash_labels = ['None-crash' for _ in none_crash_data_list]
vulner_labels = ['Vulner' for _ in vulner_data_list]

In [10]:
none_crash_train_list, none_crash_test_list = train_test_split(none_crash_data_list, 
                                                                test_size=0.1,
                                                                stratify=none_crash_labels,
                                                                random_state=seed)

vulner_train_list, vulner_test_list = train_test_split(vulner_data_list, 
                                                        test_size=0.1,
                                                        stratify=vulner_labels,
                                                        random_state=seed)

In [11]:
print(len(none_crash_train_list))
print(len(none_crash_test_list))
print(len(vulner_train_list))
print(len(vulner_test_list))

27448
3050
3409
379


In [12]:
# ex) train_list[i]: "../../dataset/data_preprocessed/None-crash\21-12-01-11-07-44_end_extract_drive26\00449.pickle"
train_list = none_crash_train_list + vulner_train_list
test_list = none_crash_test_list + vulner_test_list

random.shuffle(train_list)
random.shuffle(test_list)

In [20]:
test_list[200]

'../../dataset/data_preprocessed/None-crash\\21-12-01-14-54-56_end_extract_drive17\\00431.pickle'

In [14]:
train_labels = [path.split('/')[4].split('\\')[0] for path in train_list]

In [15]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=train_labels,
                                          random_state=seed)

In [16]:
print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

Train Data: 24685
Validation Data: 6172
Test Data: 3429


## Load Dataset

In [27]:
class lidar_dataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list # 파일명이 들어가야 함

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        lidar_path = self.file_list[idx]
        
        with open(lidar_path,"rb") as fr:
            data = pickle.load(fr)

        tensor = data['tensor']
        if data['label'] == 'None-crash':  
            label = 0
        else:
            label = 1
            
        return tensor, label

In [28]:
train_data = lidar_dataset(train_list)
valid_data = lidar_dataset(valid_list)
test_data = lidar_dataset(test_list)

In [29]:
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)

In [30]:
print(len(train_data), len(train_loader))

24685 1543


In [31]:
print(len(valid_data), len(valid_loader))

6172 386


## Effecient Attention

### Load Model


In [32]:
PATH_model = "../../model/model19.pt"
model = torch.load(PATH_model)

### Confusion Matrix

In [40]:
TP, FP, FN, TN = 0, 0, 0, 0
FN_list = list()
FP_list = list()

print("#Test-data:", len(test_list))

for i in range(len(test_list)):
    if i%50 == 0: print(i)

    with open(test_list[i],"rb") as fr:
        tmp_data = pickle.load(fr)

    test_label = tmp_data['label']
    if test_label == "None-crash":
        test_label = 0
    else:
        test_label = 1
    test_tensor = np.asarray(tmp_data['tensor'])
    test_tensor = np.asarray([test_tensor])
    test_tensor = torch.from_numpy(test_tensor)
    test_tensor = test_tensor.to(torch.float32).cuda()

    pred = model(test_tensor)
    if pred[0][0] > pred[0][1]:
        pred_idx = 0
    elif pred[0][0] < pred[0][1]:
        pred_idx = 1
    else:
        assert pred[0][0] == pred[0][1], 'Same class score'

    if pred_idx == 1 and test_label == 1:
        TP += 1
    elif pred_idx == 1 and test_label == 0:
        FP += 1
        FP_list.append(test_list[i])
    elif pred_idx == 0 and test_label == 1:
        FN += 1
        FN_list.append(test_list[i])
    elif pred_idx == 0 and test_label == 0:
        TN += 1

#Test-data: 3429
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400


In [41]:
print(TP, FP, FN, TN)

369 3 10 3047


In [44]:
for ss in FP_list:
    print(ss)

../../dataset/data_preprocessed/None-crash\21-12-01-11-07-44_end_extract_drive19\00025.pickle
../../dataset/data_preprocessed/None-crash\21-12-29-11-04-19_end_extract_drive14\00108.pickle
../../dataset/data_preprocessed/None-crash\21-12-01-11-07-44_end_extract_drive26\00317.pickle


In [45]:
for ss in FN_list:
    print(ss)

../../dataset/data_preprocessed/Vulner\21-12-30-15-25-54_end_extract_drive5\00287.pickle
../../dataset/data_preprocessed/Vulner\22-01-13-11-54-40_end_extract_drive2\00288.pickle
../../dataset/data_preprocessed/Vulner\21-12-20-14-47-47_end_extract_drive7\00298.pickle
../../dataset/data_preprocessed/Vulner\21-12-06-15-48-11_end_extract_drive2\00290.pickle
../../dataset/data_preprocessed/Vulner\21-12-14-15-04-44_end_extract_drive2\00301.pickle
../../dataset/data_preprocessed/Vulner\21-12-06-10-41-51_end_extract_drive4\00291.pickle
../../dataset/data_preprocessed/Vulner\22-01-13-16-14-44_end_extract_drive3\00304.pickle
../../dataset/data_preprocessed/Vulner\21-12-01-15-33-47_end_extract_drive4\00222.pickle
../../dataset/data_preprocessed/Vulner\21-12-30-15-25-54_end_extract_drive5\00304.pickle
../../dataset/data_preprocessed/Vulner\21-12-14-15-04-44_end_extract_drive2\00302.pickle


In [33]:
print("Recall:", TP/(TP+FN))
print("Precision:", TP/(TP+FP))
print("Accuracy:", (TP+TN)/(TP + FP + FN + TN))

Recall: 0.9736147757255936
Precision: 0.9919354838709677
Accuracy: 0.9962088072324293
